# Working with multiple files

On occasion, we will need to combine more than 2 files using some combination of `UNION` and `JOIN`.  In this lecture, we will show a clean approach to scaling up these operations up to any number of files.  In the process, we will

1. Use `list` comprehensions to process and `UNION` many similar files.
2. Use `dict` comprehensions to store and access many tables by name.

In [4]:
import polars as pl

## The Basics of working with many files.

* Use `glob.glob` to find all files that match a pattern
* Convert all files to `pd.DataFrames`
* Store the `df` in a list or dictionary

### What the heck is a `glob`

`glob.glob`

* Takes a path regular expression
* Returns a list of files that match the patterm
* Relative paths!

In [125]:
from glob import glob
sales_files = glob('./data/auto_sales_*.csv')
sales_files

['./data\\auto_sales_apr.csv', './data\\auto_sales_may.csv']

### Search tools for `glob`

* Use `*` as a wildcard,
* Use `?` for optional characters, and
* Use `[...]` to define character classes.

### Using `polars` built-in `glob`

* `pl.read_csv( ..., glob=True)` is default,
* Will search for all files when given a wildcard/optional/class, and
* UNION the resulting tables.

**Note.** All information in the file name is lost!

In [12]:
sales_files = pl.read_csv('./data/auto_sales_*.csv')
sales_files

,Salesperson,Compact,Sedan,SUV,Truck
i64,str,i64,i64,i64,i64
0,"""Ann""",22,18,15,12
1,"""Bob""",19,12,17,20
2,"""Yolanda""",19,8,32,15
3,"""Xerxes""",12,23,18,9
0,"""Ann""",22,18,15,12
1,"""Bob""",20,14,6,24
2,"""Yolanda""",19,10,28,17
3,"""Xerxes""",11,27,17,9


#### BEWARE - Files need to be UNIONABLE

In [ ]:
pl.read_csv('./data/baseball/core/*.csv')

## Store in `dict` or `list`?

* Natural sequence/order? $\rightarrow$ `list`
    *  Example: Lakes data and years are a natural sequence
* Easier to refer by name? $\rightarrow$ `dict`
    * Baseball files have no order and easier to refer to by name

## Example - Using `glob` to read and UNION the sales data

Using `glob` with a `list` to automate reading an combining files 

#### Step 1 - Get the file names

In [92]:
from glob import glob
sales_files = glob('./data/auto_sales_*.csv')
sales_files

['./data\\auto_sales_apr.csv', './data\\auto_sales_may.csv']

#### Step 2 - Read the files into a list of data frames

In [95]:
sales_by_month = [pl.read_csv(f) for f in sales_files]
sales_by_month

[shape: (4, 6)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
 │ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    │
 │ 2   ┆ Yolanda     ┆ 19      ┆ 8     ┆ 32  ┆ 15    │
 │ 3   ┆ Xerxes      ┆ 12      ┆ 23    ┆ 18  ┆ 9     │
 └─────┴─────────────┴─────────┴───────┴─────┴───────┘,
 shape: (4, 6)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
 │ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    │
 │ 2   ┆ Yolanda     ┆ 19      ┆ 1

 #### Inspect each data from with head

In [98]:
[df.head(2) for df in sales_by_month]

[shape: (2, 6)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
 │ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    │
 └─────┴─────────────┴─────────┴───────┴─────┴───────┘,
 shape: (2, 6)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
 │ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    │
 └─────┴─────────────┴─────────┴───────┴─────┴───────┘]

#### Inspecting the `shape`s

In [101]:
[df.shape for df in sales_by_month]

[(4, 6), (4, 6)]

#### Step 3 - Pull off the month from the file names with a RegEx

In [143]:
import re

MONTH_RE = re.compile(r'^\./data/auto_sales_([a-zA-Z_]*)\.csv$')
get_month = lambda p: MONTH_RE.match(p).group(1) if MONTH_RE.match(p) else None

In [147]:
sales_files = glob.glob('./data/auto_sales_*.csv')
sales_files

['./data\\auto_sales_apr.csv', './data\\auto_sales_may.csv']

In [149]:
month_names = [get_month(p) for p in sales_files]
month_names

[None, None]

In [151]:
sales_dfs = [pl.read_csv(f) for f in sales_files]

sales_dfs

[shape: (4, 6)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
 │ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    │
 │ 2   ┆ Yolanda     ┆ 19      ┆ 8     ┆ 32  ┆ 15    │
 │ 3   ┆ Xerxes      ┆ 12      ┆ 23    ┆ 18  ┆ 9     │
 └─────┴─────────────┴─────────┴───────┴─────┴───────┘,
 shape: (4, 6)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
 │ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    │
 │ 2   ┆ Yolanda     ┆ 19      ┆ 1

#### Combine month and tables in one `list` comprehension

Note that we will need the month name later, so we are storing it in a `tuple` with the data frame for now.

In [113]:
sales_by_month = [(get_month(f), pl.read_csv(f)) for f in sales_files]

sales_by_month

[(None,
  shape: (4, 6)
  ┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
  │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
  │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
  │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
  ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
  │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
  │ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    │
  │ 2   ┆ Yolanda     ┆ 19      ┆ 8     ┆ 32  ┆ 15    │
  │ 3   ┆ Xerxes      ┆ 12      ┆ 23    ┆ 18  ┆ 9     │
  └─────┴─────────────┴─────────┴───────┴─────┴───────┘),
 (None,
  shape: (4, 6)
  ┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
  │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
  │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
  │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
  ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
  │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
  │ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    

#### Step 4 - Add a month column to each file

Notice that we need to put the `polars` dot-chain *inside* the `list` comprehension to allow access to the names.

In [42]:
sale_files_with_month = [(df
                          .with_columns(month = pl.lit(mon))
                         )
                         for mon, df in sales_by_month
                        ]

In [44]:
[df.head(2) for df in sale_files_with_month]

[shape: (2, 7)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ month │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   ┆ ---   │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   ┆ null  │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╪═══════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ null  │
 │ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    ┆ null  │
 └─────┴─────────────┴─────────┴───────┴─────┴───────┴───────┘,
 shape: (2, 7)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ month │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   ┆ ---   │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   ┆ null  │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╪═══════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ null  │
 │ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    ┆ null  │
 └─────┴─────────────┴──

#### Step 5 - Combine the files using `pl.concat`

In [47]:
combined_files = pl.concat(sale_files_with_month)
combined_files

,Salesperson,Compact,Sedan,SUV,Truck,month
i64,str,i64,i64,i64,i64,null
0,"""Ann""",22,18,15,12,null
1,"""Bob""",19,12,17,20,null
2,"""Yolanda""",19,8,32,15,null
3,"""Xerxes""",12,23,18,9,null
0,"""Ann""",22,18,15,12,null
1,"""Bob""",20,14,6,24,null
2,"""Yolanda""",19,10,28,17,null
3,"""Xerxes""",11,27,17,9,null


## <font color="red"> Exercise 3.1</font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Use `glob` to get all 6 file paths.
2. Use a regular expression to create a `lambda` function that pulls the month from the files.
3. Read the 6 data frames into a `list` of `tuples` containing the month name corresponding data frame.
4. Add the month column each data frame using a pipe inside of a comprehension.
5. Use `pd.concat` to combine these 6 data frames into one combined `df`

In [161]:
file_paths = glob.glob('./data/uber-raw-data-*.csv')  

MONTH_RE = re.compile(r'./data/uber-raw-data-([a-z]{3})(\d{2})\.csv$')

get_month = lambda p: MONTH_RE.match(p).group(1) if MONTH_RE.match(p) else None

data_frames = [(get_month(file), pl.read_csv(file)) for file in file_paths]

data_frames_with_month = [
                            (month, df.with_columns(pl.lit(month).alias("Month"))) 
                            for month, df in data_frames
                         ]

combined_df = pl.concat(df for month, df in data_frames_with_month)

combined_df

Date/Time,Lat,Lon,Base,Month
str,f64,f64,str,null
"""4/18/2014 21:38:00""",40.7359,-73.9852,"""B02682""",null
"""4/23/2014 15:19:00""",40.7642,-73.9543,"""B02598""",null
"""4/10/2014 7:15:00""",40.7138,-74.0103,"""B02598""",null
"""4/11/2014 15:23:00""",40.7847,-73.9698,"""B02682""",null
"""4/7/2014 17:26:00""",40.646,-73.7767,"""B02598""",null
…,…,…,…,…
"""9/13/2014 16:51:00""",40.7959,-73.9326,"""B02617""",null
"""9/22/2014 22:48:00""",40.7541,-73.949,"""B02617""",null
"""9/11/2014 22:38:00""",40.7292,-73.999,"""B02512""",null
